In [2]:
import pandas as pd 
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

from datetime import datetime

In [3]:
from sklearn.feature_selection import SelectFromModel
from prophet.utilities import regressor_coefficients

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb

# Importing DATA

In [4]:
#group = ['Banking','United states','United states Banking']
group = ['Banking']

if group == ['Banking']:
    df_b_cluster = pd.read_csv(r'.\Banking\clusters_result.csv')
    df_b_outliers = pd.read_csv(r'.\Banking\df_outliers_results.csv')
    #Banking cluster
    df_cluster0 = list(df_b_cluster['Busns Id'][(df_b_cluster['gmm_3'] == 0)])
    df_cluster1 = list(df_b_cluster['Busns Id'][(df_b_cluster['gmm_3'] == 1)])
    df_cluster2 = list(df_b_cluster['Busns Id'][(df_b_cluster['gmm_3'] == 2)])
    #Banking outliers
    df_outliers = list(df_b_outliers['Busns Id'][(df_b_outliers['ecod'] == True)])
elif group == ['United states']:
    df_us_cluster = pd.read_csv(r'.\United States\clusters_result.csv')
    df_us_outliers = pd.read_csv(r'.\United States\df_outliers_results.csv')
    #United states clusters
    df_cluster0 = list(df_us_cluster['Busns Id'][(df_us_cluster['gmm_3'] == 0)])
    df_cluster1 = list(df_us_cluster['Busns Id'][(df_us_cluster['gmm_3'] == 1)])
    df_cluster2 = list(df_us_cluster['Busns Id'][(df_us_cluster['gmm_3'] == 2)])
    #United states outliers
    df_outliers = list(df_us_outliers['Busns Id'][(df_us_outliers['ecod'] == True)])
elif group == ['United states Banking']:
    df_usb_cluster = pd.read_csv(r'.\United States Banking\clusters_result.csv')
    df_usb_outliers = pd.read_csv(r'.\\United States Banking\\df_outliers_results.csv')
    #United states Banking clusters
    df_cluster0 = list(df_usb_cluster['Busns Id'][(df_usb_cluster['gmm_3'] == 0)])
    df_cluster1 = list(df_usb_cluster['Busns Id'][(df_usb_cluster['gmm_3'] == 1)])
    df_cluster2 = list(df_usb_cluster['Busns Id'][(df_usb_cluster['gmm_3'] == 2)])
    #United states Banking outliers
    df_outliers = list(df_usb_outliers['Busns Id'][(df_usb_outliers['ecod'] == True)])
    

## Cluster allocation

In [5]:
#cluster = ['0' , '1' , '2' , 'outlier' , 'all']
cluster = ['all']
if cluster == ['0']:
    sep_acc = df_cluster0
elif cluster == ['1']:
    sep_acc = df_cluster1
elif cluster == ['2']:
    sep_acc = df_cluster2
elif cluster == ['outlier']:
    sep_acc = df_outliers
elif cluster == ['all']:
    sep_acc = df_outliers + df_cluster0 + df_cluster1 + df_cluster2

In [6]:
#sep_acc = ['EARLY_WARN_S','TD_BANK_NA']
final_list = sep_acc
len(final_list)

395

In [7]:
#loading the data
df = pd.read_csv('all_factors_matrix_in_time.csv')
df['DATE'] = pd.to_datetime(df[['YEAR','MONTH']].assign(DAY = 1))

C:\Users\AishaniSanjayPatil\AppData\Local\Temp\ipykernel_13128\2532084808.py:2: DtypeWarning: Columns (12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_factors_matrix_in_time.csv')


In [8]:
regressor_list =  ['profit_per_fte','labor_charge_hourly','labor_cost','vended_services_cost','inhouse_labor_ratio','labor_band','profit_per_labor_dolar',
                         'managed_contracts_rev','consult_contracts_rev','managed_contracts_cost','consult_contracts_cost']

#regressor_list = ['profit_per_fte','vended_services_cost','profit_per_labor_dolar','managed_contracts_cost','consult_contracts_cost']

## Final dataframe

In [9]:
#extract required data for prediction from the available data
df1 = df[['Busns Id','DATE','gross_profit_margin'] + regressor_list]
#drop unwanted values
df2 = df1.replace(['Nan','nan','#NAME?','NaN','inf','-inf','', ' ',np.inf, -np.inf], None)

#df2[['DATE','gross_profit_margin'] + regressor_list] = df2.groupby('Busns Id').fillna(method = 'ffill')
#df2[['DATE','gross_profit_margin'] + regressor_list] = df2.groupby('Busns Id').fillna(method = 'bfill')

df2['gross_profit_margin'] = df2['gross_profit_margin'].astype(float).round(decimals = 2)

# Function for predicting regressors and using models to find out the best fitting subsets of regressors

In [16]:
def compare_diff_models(df_func, acc_list,regressor_list):
    #df_func = df_func.set_index('Busns Id' , inplace= True)
    for acc in acc_list:
        
        print(acc)
        df_acc = df_func.loc[df_func['Busns Id'] == acc]
        remove_column = df_acc.columns[df_acc.isna().all()]
        for col in remove_column:
            if col in regressor_list:
                regressor_list.remove(col)
        df_acc = df_acc.sort_values('DATE')
        df_acc.reset_index()
        #print(df_acc)
        #predicting values of regressors
        df_sub = df_acc[['DATE','gross_profit_margin']]
        df_sub = df_sub.reset_index()
        df_sub = df_sub.drop(['index'],axis = 1)
        for i in regressor_list:
            df_reg_i = df_acc[['DATE',i]]
            df_reg_i.columns = ['ds' , 'y']
            train_reg_i = df_reg_i.iloc[:8]
            model_reg_i = Prophet()
            model_reg_i.fit(train_reg_i)
            future_reg_i = pd.DataFrame(df_acc['DATE'].iloc[8:])
            future_reg_i.columns = ['ds']
            future_reg_i['ds'] = pd.to_datetime(future_reg_i['ds'])
            forecast_reg_i = model_reg_i.predict(future_reg_i)
            predicted_reg_i = pd.DataFrame()
            predicted_reg_i[i] = forecast_reg_i['yhat']
            df_sub[i] = pd.concat([df_acc[i].iloc[:8] , predicted_reg_i[i]] , axis = 0, ignore_index = True)
        
        #prophet model to find the best regressors
        df_sub_pr = df_sub.rename(columns = {'DATE' : 'ds','gross_profit_margin' : 'y'})
        #print(df_sub)
        train_pr = df_sub_pr.iloc[:8]
        test_pr = df_sub_pr.iloc[8:]
        model_pr = Prophet(interval_width = 0.3) #call the object  
        for j in regressor_list: #add regressor
            model_pr.add_regressor(j)
        #fit the model_pr
        model_pr.fit(train_pr)
        regressor_coef_pr = regressor_coefficients(model_pr)
        regressor_coef_pr['coef'] = regressor_coef_pr['coef'].abs()
        regressor_coef_pr = regressor_coef_pr[['regressor','coef']].sort_values('coef',ascending=False).reset_index()
        important_features_pr = list(regressor_coef_pr['regressor'].loc[:3])
        
        #Train test data split
        df_sub_reg = df_sub.copy()
        df_sub_reg['y'] = df_sub_reg['gross_profit_margin'].shift(1)
        df_sub_reg['y'] = df_sub_reg['y'].bfill()
        #print(df_sub_reg)
        regressor_list_new = regressor_list + ['gross_profit_margin']
        df_sub_reg[regressor_list_new] = df_sub_reg[regressor_list_new].apply(pd.to_numeric, errors='coerce')      
        df_sub_reg.set_index('DATE', inplace=True)
        X = df_sub_reg[regressor_list_new]
        y = df_sub_reg['y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False)
        #print(X)
        #print(X_train)

        #Random forest regressor
        rf_model = RandomForestRegressor(n_estimators=100)
        rf_model.fit(X_train, y_train)
        sfm_rfr = SelectFromModel(rf_model)
        sfm_rfr.fit(X_train, y_train)
        important_features_RFR_1 = list(X.columns[sfm_rfr.get_support()])
        feature_importance_df = pd.DataFrame({'Feature': regressor_list_new,'Importance': rf_model.feature_importances_})
        important_features_RFR_2 = feature_importance_df.head(5)['Feature'].tolist()

        #Decision Tree
        reg_dt = DecisionTreeRegressor(random_state=42)
        reg_dt.fit(X_train, y_train)
        sfm_dt = SelectFromModel(reg_dt)
        sfm_dt.fit(X_train, y_train)
        important_features_dt = list(X.columns[sfm_dt.get_support()])

        #XG Boost
        model_xg = xgb.XGBRegressor()
        model_xg.fit(X_train, y_train)
        sfm_xg = SelectFromModel(model_xg)
        sfm_xg.fit(X_train, y_train)
        important_features_xg_1 = list(X.columns[sfm_xg.get_support()])
        importances_xg = model_xg.get_booster().get_score(importance_type='weight')
        importances_xg = sorted(importances_xg.items(), key=lambda x: x[1], reverse=True)
        important_features_xg_2 = [feature for feature, _ in importances_xg]
        #if 'gross_profit_margin' in important_features_xg_2:
        #    important_features_xg_2.remove('gross_profit_margin')
        best_fit_dict = {'Prophet':important_features_pr, 'Random Forest 1': important_features_RFR_1, 'Random Forest 2': important_features_RFR_2,
                         'Decision Tree':important_features_dt,'XG Boost 1': important_features_xg_1,'XG Boost 2': important_features_xg_2}

        #best_fit_list = [important_features_pr,important_features_RFR_1,important_features_RFR_2,important_features_dt,important_features_xg_1,important_features_xg_2]
        print(f'Prophet: {important_features_pr}\nRandom forest (Select from model){important_features_RFR_1}\nRandom forest (feature importance function): {important_features_RFR_2}\nDecision Tree (Select from model): {important_features_dt}\nXG Boost (Select from model):{important_features_xg_1}\nXG Boost (Get booster function): {important_features_xg_2}')

        for model,best_fit_list in best_fit_dict.items():
            #print(type)
            if df_sub['gross_profit_margin'].iloc[8:].values.all() is not np.isnan(df_sub['gross_profit_margin'].iloc[8:]).any():

                list_init = [acc , model , best_fit_list]

                make_forecast(df_sub , best_fit_list , acc , list_init)

                all_metric_df.loc[len(all_metric_df)] = list_init
        

# Prophet function

In [11]:
def make_forecast(df_account , regressor_best_list , acc , list_init):

    if 'gross_profit_margin' in regressor_best_list:
        regressor_best_list.remove('gross_profit_margin')

    df_acc = df_account.rename(columns = {'DATE' : 'ds','gross_profit_margin' : 'y'})
    train = df_acc.iloc[:8]
    test = df_acc.iloc[8:]
    model = Prophet(interval_width = 0.3)
    for i in regressor_best_list:
        model.add_regressor(i)
    model.fit(train)
    #make future dataframe
    future = pd.DataFrame(test.ds)
    future.columns = ['ds']
    future['ds'] = pd.to_datetime(future['ds'])
    #defining future regressor dataframe
    for k in regressor_best_list:
        future[k] = df_acc[k].iloc[8:]
    forecast = model.predict(future)
    
    y_pred = forecast.iloc[-4:]['yhat']
    y_true = test['y']
    if y_true.values.all() is not np.isnan(y_true).any():
        #mae_dict = {'Business Id' : acc}
        for m in range(1,len(y_pred)+1):
            y_pred_m = y_pred[:m]
            y_true_m = y_true[:m]
            #print(y_true_m,y_pred_m)
            mae = mean_absolute_error(y_true_m,y_pred_m).round()
            #mae_dict[f'MAE_for_month_{m}'] = mae
            list_init.append(mae)
            #print(f'For {i} account:\nMean absolute error for {m} month/s prediction: {mae} (GPM %)')
            mape = mean_absolute_percentage_error(y_true_m,y_pred_m).round(decimals = 2)
            #mae_dict[f'MAPE_for_month_{m}'] = mape
            list_init.append(mape)
            #print(f'For {i} account:\nMean absolute percentage error for {j} month/s prediction: {mape} (GPM %)')
        #print(list_init)
        return list_init


In [113]:
all_metric_df

,Account,Feature_select_model,list_of_regressors,MAE1,MAPE1,MAE2,MAPE2,MAE3,MAPE3,MAE4,MAPE4
0,BANCO_SANT_C,Prophet,"[managed_contracts_cost, managed_contracts_rev...",11.0,0.22,12.0,0.22,19.0,0.27,36.0,0.93
1,BANCO_SANT_C,Random Forest 1,"[labor_band, profit_per_labor_dolar, consult_c...",10.0,0.20,11.0,0.20,18.0,0.25,36.0,0.93
2,BANCO_SANT_C,Random Forest 2,"[profit_per_fte, labor_charge_hourly, labor_co...",11.0,0.22,12.0,0.22,19.0,0.27,36.0,0.93
3,BANCO_SANT_C,Decision Tree,"[labor_band, consult_contracts_rev]",11.0,0.22,12.0,0.22,19.0,0.27,36.0,0.93
4,BANCO_SANT_C,XG Boost 1,"[labor_band, profit_per_labor_dolar]",11.0,0.22,12.0,0.22,19.0,0.27,37.0,0.93
5,BANCO_SANT_C,XG Boost 2,"[profit_per_fte, profit_per_labor_dolar, labor...",12.0,0.22,13.0,0.23,19.0,0.28,37.0,0.93
6,BK_AMER_NAT,Prophet,"[managed_contracts_rev, vended_services_cost, ...",155.0,161.64,203.0,91.25,254.0,63.99,321.0,49.00
7,BK_AMER_NAT,Random Forest 1,"[profit_per_fte, managed_contracts_rev, manage...",6.0,5.84,16.0,4.05,31.0,3.24,64.0,2.74
8,BK_AMER_NAT,Random Forest 2,"[profit_per_fte, labor_charge_hourly, labor_co...",16.0,16.86,28.0,10.05,43.0,7.36,76.0,5.86
9,BK_AMER_NAT,Decision Tree,"[profit_per_labor_dolar, managed_contracts_rev...",6.0,6.47,17.0,4.40,32.0,3.48,65.0,2.92


In [ ]:
save_test_result = 

In [118]:
df_prophet = all_metric_df.loc[all_metric_df['Feature_select_model'] == 'Prophet']
df_prophet

,Account,Feature_select_model,list_of_regressors,MAE1,MAPE1,MAE2,MAPE2,MAE3,MAPE3,MAE4,MAPE4
0,BANCO_SANT_C,Prophet,"[managed_contracts_cost, managed_contracts_rev...",11.0,0.22,12.0,0.22,19.0,0.27,36.0,0.93
6,BK_AMER_NAT,Prophet,"[managed_contracts_rev, vended_services_cost, ...",155.0,161.64,203.0,91.25,254.0,63.99,321.0,49.00
12,MASTER_TECH,Prophet,"[consult_contracts_rev, consult_contracts_cost...",510.0,3.01,476.0,11.38,457.0,9.40,461.0,8.25
18,NAVY_FED_CRE,Prophet,"[vended_services_cost, inhouse_labor_ratio, co...",31.0,1.75,32.0,1.71,25.0,1.29,20.0,1.01
24,RBC_CAPI_MAR,Prophet,"[managed_contracts_cost, managed_contracts_rev...",85.0,1.42,75.0,1.04,81.0,1.29,82.0,1.55
30,US_WELLSFARG,Prophet,"[managed_contracts_cost, vended_services_cost,...",323.0,1.20,191.0,6.22,128.0,4.15,104.0,3.54


In [117]:
df_prophet.mean()

C:\Users\AishaniSanjayPatil\AppData\Local\Temp\ipykernel_52420\1045645465.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_prophet.mean()


MAE1     185.833333
MAPE1     28.206667
MAE2     164.833333
MAPE2     18.636667
MAE3     160.666667
MAPE3     13.398333
MAE4     170.666667
MAPE4     10.713333
dtype: float64

# Final Run

In [17]:
all_metric_df = pd.DataFrame(columns = ['Account' , 'Feature_select_model' , 'list_of_regressors','MAE1','MAPE1','MAE2','MAPE2','MAE3','MAPE3','MAE4','MAPE4'])
compare_diff_models(df2,final_list,regressor_list)

AMEXPRESS


16:32:11 - cmdstanpy - INFO - Chain [1] start processing
16:32:11 - cmdstanpy - INFO - Chain [1] done processing
16:32:12 - cmdstanpy - INFO - Chain [1] start processing
16:32:12 - cmdstanpy - INFO - Chain [1] done processing
16:32:13 - cmdstanpy - INFO - Chain [1] start processing
16:32:16 - cmdstanpy - INFO - Chain [1] done processing


Prophet: ['managed_contracts_cost', 'managed_contracts_rev', 'labor_charge_hourly', 'profit_per_labor_dolar']
Random forest (Select from model)['managed_contracts_rev', 'managed_contracts_cost', 'gross_profit_margin']
Random forest (feature importance function): ['profit_per_fte', 'labor_charge_hourly', 'labor_cost', 'vended_services_cost', 'inhouse_labor_ratio']
Decision Tree (Select from model): ['managed_contracts_rev', 'managed_contracts_cost', 'gross_profit_margin']
XG Boost (Select from model):['managed_contracts_rev', 'managed_contracts_cost']
XG Boost (Get booster function): ['managed_contracts_rev', 'managed_contracts_cost', 'gross_profit_margin']


16:32:17 - cmdstanpy - INFO - Chain [1] start processing
16:32:18 - cmdstanpy - INFO - Chain [1] done processing
16:32:18 - cmdstanpy - INFO - Chain [1] start processing
16:32:19 - cmdstanpy - INFO - Chain [1] done processing
16:32:19 - cmdstanpy - INFO - Chain [1] start processing
16:32:19 - cmdstanpy - INFO - Chain [1] done processing
16:32:20 - cmdstanpy - INFO - Chain [1] start processing
16:32:20 - cmdstanpy - INFO - Chain [1] done processing
16:32:21 - cmdstanpy - INFO - Chain [1] start processing
16:32:21 - cmdstanpy - INFO - Chain [1] done processing
16:32:22 - cmdstanpy - INFO - Chain [1] start processing
16:32:22 - cmdstanpy - INFO - Chain [1] done processing


ARGBANCOSANT


...

Prophet: ['consult_contracts_cost', 'managed_contracts_cost', 'labor_cost', 'managed_contracts_rev']
Random forest (Select from model)['profit_per_fte', 'labor_charge_hourly', 'labor_band', 'managed_contracts_cost', 'consult_contracts_cost', 'gross_profit_margin']
Random forest (feature importance function): ['profit_per_fte', 'labor_charge_hourly', 'labor_cost', 'vended_services_cost', 'inhouse_labor_ratio']
Decision Tree (Select from model): ['labor_band', 'consult_contracts_cost']
XG Boost (Select from model):['inhouse_labor_ratio', 'labor_band']
XG Boost (Get booster function): ['profit_per_fte', 'vended_services_cost', 'consult_contracts_cost', 'labor_band', 'inhouse_labor_ratio', 'labor_cost', 'labor_charge_hourly', 'managed_contracts_cost']


16:39:19 - cmdstanpy - INFO - Chain [1] start processing
16:39:22 - cmdstanpy - INFO - Chain [1] done processing
16:39:23 - cmdstanpy - INFO - Chain [1] start processing
16:39:26 - cmdstanpy - INFO - Chain [1] done processing
16:39:26 - cmdstanpy - INFO - Chain [1] start processing
16:39:26 - cmdstanpy - INFO - Chain [1] done processing
16:39:26 - cmdstanpy - INFO - Chain [1] start processing
16:39:26 - cmdstanpy - INFO - Chain [1] done processing
16:39:26 - cmdstanpy - INFO - Chain [1] start processing
16:39:27 - cmdstanpy - INFO - Chain [1] done processing
16:39:27 - cmdstanpy - INFO - Chain [1] start processing
16:39:27 - cmdstanpy - INFO - Chain [1] done processing


UKDAIWA


16:39:27 - cmdstanpy - INFO - Chain [1] start processing
16:39:28 - cmdstanpy - INFO - Chain [1] done processing
16:39:28 - cmdstanpy - INFO - Chain [1] start processing
16:39:28 - cmdstanpy - INFO - Chain [1] done processing
16:39:28 - cmdstanpy - INFO - Chain [1] start processing
16:39:29 - cmdstanpy - INFO - Chain [1] done processing
16:39:30 - cmdstanpy - INFO - Chain [1] start processing
16:39:30 - cmdstanpy - INFO - Chain [1] done processing
16:39:30 - cmdstanpy - INFO - Chain [1] start processing
16:39:30 - cmdstanpy - INFO - Chain [1] done processing
16:39:31 - cmdstanpy - INFO - Chain [1] start processing
16:39:33 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values